In [1]:
from math import sqrt
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [2]:
import numpy as np
from scipy.stats import norm

S0 = 1.00
K = 1.08
T = 1
num_steps = 100
sigma = 0.2
mu = 0.08
r = 0.04
n_path = 10_000


def gbm(S0, r, sigma, t, n_path):
    drift = r * t
    time_steps = np.diff(t, prepend=0)
    normals = np.random.standard_normal((n_path, num_steps+1))
    normals = np.vstack([normals, -normals])
    vol = sigma * np.sqrt(time_steps) * normals.cumsum(axis=1)
    exponent = drift + vol
    paths = S0 * np.exp(exponent)
    return paths

t = np.linspace(0, T, num_steps + 1)
S = gbm(S0, r, sigma, t, n_path)
S


array([[1.        , 0.97846052, 0.96327748, ..., 1.13962736, 1.14796697,
        1.1264337 ],
       [1.        , 1.01046427, 1.0196638 , ..., 1.22213225, 1.20051055,
        1.22044441],
       [1.        , 1.03108928, 0.97927927, ..., 0.67790022, 0.67179563,
        0.6848563 ],
       ...,
       [1.        , 1.01796355, 0.99472122, ..., 0.67379088, 0.69099357,
        0.67930643],
       [1.        , 1.02677579, 1.03018634, ..., 1.50266239, 1.5165647 ,
        1.52515785],
       [1.        , 1.02704319, 1.03201361, ..., 0.99062583, 1.0149136 ,
        1.00786018]])

In [3]:
def _d1(S, K, r, sigma, t, T):
    t2m = T - t
    numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
    denominator = sigma * np.sqrt(t2m)
    with np.errstate(divide='ignore'):
        return numerator / denominator

def _d2(d1, sigma, t, T):
    t2m = T - t
    return d1 - sigma * np.sqrt(t2m)

def call(S, K, r, t, T, d1, d2):
    return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)

def put(S, K, r, t, T, d1, d2):
    return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)

def call_delta(d1):
    return norm.cdf(d1)

def put_delta(d1):
    return norm.cdf(d1) - 1

d1 = _d1(S, K, r, sigma, t, T)
d2 = _d2(d1, sigma, t, T)

c = call(S, K, r, t, T, d1, d2)
p = put(S, K, r, t, T, d1, d2)

In [4]:
print(f'Call={c[0, 0]}')
print(f'Put={p[0, 0]}')

Call=0.06370611880864996
Put=0.10135871309315903


In [5]:
delta = call_delta(d1)[:, :-1]
delta

array([[4.66208127e-001, 4.21651539e-001, 3.89472115e-001, ...,
        9.08260088e-001, 9.73957632e-001, 9.98970454e-001],
       [4.66208127e-001, 4.85666760e-001, 5.02643464e-001, ...,
        9.99739833e-001, 9.99994917e-001, 9.99999948e-001],
       [4.66208127e-001, 5.26152891e-001, 4.21725141e-001, ...,
        1.60934264e-042, 6.53266899e-061, 1.48712238e-124],
       ...,
       [4.66208127e-001, 5.00487301e-001, 4.52851757e-001, ...,
        1.15797298e-045, 1.84574251e-062, 1.87704687e-110],
       [4.66208127e-001, 5.17761426e-001, 5.23317267e-001, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       [4.66208127e-001, 5.18282919e-001, 5.26881028e-001, ...,
        4.75884657e-003, 1.29959934e-003, 1.04240208e-003]])

In [6]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(S[:, -1] - K, 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)

np.mean(costs)

0.06405721399566709

In [7]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta - 1, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(K - S[:, -1], 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)
np.mean(costs)

0.10170980828017626

In [8]:
from typing import Protocol, Sequence
from dataclasses import dataclass

class Stock:
    def __init__(
        self, S0: float, mu: float, r: float, 
        sigma: float, time_steps: Sequence[float], 
        cost: float = 0.0
    ) -> None:
        self.S0 = S0
        self.mu = mu
        self.r = r
        self.sigma = sigma
        self.time_steps = time_steps
        self.cost = cost

        self.prices: np.ndarray | None = None

    def simulate(self, num_sims: int) -> None:
        drift = self.mu * self.time_steps
        time_increments = np.diff(self.time_steps, prepend=0)
        normals = np.random.standard_normal((num_sims // 2, len(self.time_steps)))
        normals = np.vstack([normals, -normals])
        vol = self.sigma * np.sqrt(time_increments) * normals.cumsum(axis=1)
        exponent = drift + vol
        self.prices = self.S0 * np.exp(exponent)

    def discount_factors(self) -> None:
        discount_factors = np.exp(-self.r * self.time_steps)
        return discount_factors

    def prices_at(self, t: float):
        return self.prices[:, self.time_steps == t].squeeze()

@dataclass
class EuropeanOption:
    stock: Stock
    strike: float
    maturity: float
    is_call: bool

    def payout(self):
        S = self.stock.prices_at(self.maturity)
        if self.is_call:
            return np.maximum(S - self.strike, 0)
        return np.maximum(self.strike - S, 0)

class HedgingStrategy(Protocol):

    @property
    def stock(self) -> Stock:
        ...
    
    @property
    def derivative(self) -> EuropeanOption:
        ...
    
    def get_hedge_ratio(self, time_step_idx: int | None = None):
        ...

@dataclass
class BlackScholesHedgingStrategy:

    option: EuropeanOption

    @property
    def stock(self) -> Stock:
        return self.option.stock
    
    @property
    def derivative(self) -> EuropeanOption:
        return self.option
    
    def get_hedge_ratio(self, time_step_idx: int | None = None):
        if time_step_idx:
            raise NotImplementedError('TODO')

        stock = self.option.stock
        S = stock.prices
        r = stock.r
        K = self.option.strike
        d = BlackScholesHedgingStrategy.d1(S, K, stock.r, stock.sigma, stock.time_steps, self.option.maturity)

        if self.option.is_call:
            return BlackScholesHedgingStrategy.call_delta(d)
        
        return BlackScholesHedgingStrategy.put_delta(d)

    @staticmethod
    def d1(S, K, r, sigma, t, T):
        t2m = T - t
        numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
        denominator = sigma * np.sqrt(t2m)

        with np.errstate(divide='ignore'):
            return numerator / denominator

    @staticmethod
    def d2(d1, sigma, t, T):
        t2m = T - t
        return d1 - sigma * np.sqrt(t2m)
    
    @staticmethod
    def call_price(S, K, r, t, T, d1, d2):
        return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)
    
    @staticmethod
    def put_price(S, K, r, t, T, d1, d2):
        return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)
    
    @staticmethod
    def call_delta(d1):
        return norm.cdf(d1)
    
    @staticmethod
    def put_delta(d1):
        return norm.cdf(d1) - 1

class Pricer:
    def price(self, hedging_strategy: HedgingStrategy) -> float:
        S = hedging_strategy.stock.prices
        discount_factors = hedging_strategy.stock.discount_factors()
        delta = hedging_strategy.get_hedge_ratio()[:, :-1]
        txns = S * np.diff(delta, axis=1, prepend=0, append=0)
        
        payoff = hedging_strategy.option.payout()
        txns[:, -1] += payoff
        disounted_txns = discount_factors * txns
        costs = disounted_txns.sum(axis=1)
        
        return np.mean(costs)

In [18]:
import numpy as np

S0 = 100
K = 108
T = 1
num_steps = 100
sigma = 0.2
mu = 0.04
r = 0.04
n_path = 10_000
t = np.linspace(0, T, num_steps + 1)


stock = Stock(S0=S0, mu=r, r=r, sigma=sigma, time_steps=t)
stock.simulate(n_path)
stock.prices

array([[100.        ,  99.99302716, 103.32715823, ..., 136.17864698,
        133.96861451, 137.38385209],
       [100.        ,  96.51449031,  98.82272838, ...,  79.82666312,
         81.87699973,  78.88146117],
       [100.        ,  97.18930799,  95.62807912, ..., 106.35660159,
        111.23171014, 107.42987526],
       ...,
       [100.        ,  99.27501184,  98.06910713, ...,  62.24882447,
         61.98195361,  62.67395644],
       [100.        , 102.2799254 , 103.69149214, ...,  82.47313416,
         81.6124057 ,  79.74215945],
       [100.        , 100.54591986,  98.30520855, ..., 101.77117655,
        100.79495815, 102.63506443]])

In [24]:
call_option = EuropeanOption(stock, K, T, is_call=True)
bs_strategy = BlackScholesHedgingStrategy(call_option)

pricer = Pricer()
pricer.price(bs_strategy)

6.410705501540723